### Training all the models for main_project_notebook.


This notebook contains the code for training of the baseline CNN, autoencoder and the CNN with augmented data. The model weights are safed in a .hdf5 format and loaded into the main_project_notebook.ipynb file. The files that can be obtained using this code are:
- `cnn_baseline_weights.hdf5`
- `autoencoder_weights.hdf5`
- `cnn_augmented_0.25_weights.hdf5`
- `cnn_augmented_0.50_weights.hdf5`
- `cnn_augmented_0.75_weights.hdf5`
- `cnn_augmented_1_weights.hdf5`

We provided them in the zip-file so it is not necessary to train all models.

Firstly, the required libraries are imported.

In [ ]:
# Load the functions and classes from main_util.py
from main_util import get_pcam_generators
from main_util import Model_architecture
from main_util import Model_transform

# Modelcheckpoint and tensorboard callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

# Other libraries
import matplotlib.pyplot as plt
import os

### Instantiating data generators

The PatchCAMELYON dataset is too big to fit in the working memory of most personal computers. This is why, we need to define some functions that will read the image data batch by batch, so only a single batch of images needs to be stored in memory at one time point. We can use the handy ImageDataGenerator function from the Keras API to do this. Note that the generators are defined within the function `get_pcam_generators` that returns them as output arguments. This function will later be called from the main code body. The function is located in `main_util.py`.

**Before executing the code block below, do not forget to change the path where the PatchCAMELYON dataset is located (that is, the location of the folder that contains `train+val` that you previously downloaded and unpacked).**

If everything is correct, the following output will be printed on screen after executing the code block:

`Found 144000 images belonging to 2 classes.`

`Found 16000 images belonging to 2 classes.`

In [ ]:
# Initialize dataset path
path = '../data'

# Call function from main_util to load generators
train_gen, val_gen = get_pcam_generators(path, 
                                         train_batch_size=16, 
                                         val_batch_size=16,
                                         class_mode="binary")

### Training the baseline CNN model

In the first cell, the model name and necessary paths are defined first. The second cell builds the CNN model architecture using the `Model_architecture` class, which is located in `main_util.py`. The third cell initializes the training phase of the model. During training, the weights are stored into a .hdf5 file.

In [ ]:
# Defining model name and filepath for the weights
model_name = "cnn_baseline"
weights_filepath = f"trained_models/{model_name}_weights.hdf5"

In [ ]:
# Class instance is made
model_cnn = Model_architecture()
model_cnn.create_cnn(kernel_size=(3,3), pool_size=(4,4), first_filters=32, second_filters=64)
model_cnn.compile_cnn(learning_rate=0.001)

# Prints a summary of the model structure
model_cnn.summary();

In [ ]:
# Define the model checkpoint and tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join("logs", model_name))
callbacks_list = [checkpoint, tensorboard]

# Train the model
train_steps = train_gen.n//train_gen.batch_size//10
val_steps = val_gen.n//val_gen.batch_size//10

history = model_cnn.fit(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=30,
                        callbacks=callbacks_list)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(20,7))

# Make plot for accuracy
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')
ax[0].set_ylim([0, 1])

# Make plot for loss
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')
ax[1].set_ylim([0, 1]);

### Training autoencoder model
First, the model name and the filepath for the saved model are defined. Next, we need to construct new data generators. The training process of the autoencoder is unsupervised so the class mode of the data generators should be set to `input`. With these new generators, the autoencoder can be trained effectively. In the third cell, the model architecture is loaded. After this is done, the training phase of the autoencoder is initialized.

In [ ]:
# Defining model name and paths
model_name = "autoencoder"
weights_filepath = f"trained_models/{model_name}_weights.hdf5"

In [ ]:
# Constructing the data generators for unsupervised learning
train_gen_ae, val_gen_ae = get_pcam_generators(path, 
                                               train_batch_size=16, 
                                               val_batch_size=16, 
                                               class_mode="input") 

In [ ]:
# Class instance is made
model_ae = Model_architecture()
model_ae.create_autoencoder(kernel_size=(3,3), pool_size=(2,2), first_filters=32, second_filters=16)
model_ae.compile_autoencoder(learning_rate=0.001)

# Prints a summary of the model structure
model_ae.summary();

In [ ]:
# Define the Tensorboard callback
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join("logs", model_name))
callbacks_list = [checkpoint, tensorboard]

# Train the model
train_steps = train_gen_ae.n//train_gen_ae.batch_size
val_steps = val_gen_ae.n//val_gen_ae.batch_size

history = model_ae.fit(train_gen_ae, steps_per_epoch=train_steps,
                        validation_data=val_gen_ae,
                        validation_steps=val_steps,
                        epochs=3,
                        callbacks=callbacks_list)

### Training CNN with augmented data
In the code below the variable `AUGMENTATION_FACTOR` can be changed to adapt the amount of augemented data used for training. For example, 0.75 means 75% augmented data and 25% original data. The generators for the augmented data are initialized with the autoencoder model as preprocessing function. This is done with the class `Model_transform`. This class is located in `main_util.py` and is responsible for augmenting the input of the data generators. 

In [ ]:
 # Define augmentation factor: choose 0.25 or 0.50 or 0.75 or 1
AUGMENTATION_FACTOR = 0.75 

# Defining model name and paths
model_name = "cnn_augmented_" + str(AUGMENTATION_FACTOR)
weights_filepath = f"trained_models/{model_name}_weights.hdf5"

In [ ]:
# Constructing the data generators for the augmented dataset  
transformation = Model_transform(ae_model=model_ae, augmentation_factor=AUGMENTATION_FACTOR)
train_gen_aug, val_gen_aug = get_pcam_generators(path, 
                                                 train_batch_size=16,
                                                 val_batch_size=16,
                                                 class_mode="binary", 
                                                 prep_function=transformation.model_transform)

In [ ]:
# Class instance is made
model_cnn_aug = Model_architecture()
model_cnn_aug.create_cnn(kernel_size=(3,3), pool_size=(4,4), first_filters=32, second_filters=64)
model_cnn_aug.compile_cnn(learning_rate=0.001)

# Prints a summary of the model structure
model_cnn_aug.summary()

In [ ]:
# Define the Tensorboard callback
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join("logs", model_name))
callbacks_list = [checkpoint, tensorboard]

# Train the model
train_steps = train_gen_aug.n//train_gen_aug.batch_size//10
val_steps = val_gen_aug.n//val_gen_aug.batch_size//10

history = model_cnn_aug.fit(train_gen_aug, steps_per_epoch=train_steps,
                        validation_data=val_gen_aug,
                        validation_steps=val_steps,
                        epochs=30, 
                        callbacks=callbacks_list)
                            

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(20,7))

# Make plot for accuracy
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')
ax[0].set_ylim([0, 1])

# Make plot for loss
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')
ax[1].set_ylim([0, 1]);

This notebook can be used to obtain all the trained models, which are needed to run `main_project_notebook.ipynb` and `domain_generalization.ipynb`.